In [ ]:
# chr(0)
# print(chr(0))
# # "this is a test" + chr(0) + "string"
# print("this is a test" + chr(0) + "string")

# # Output: this is a teststring

In [5]:
# UTF 8 Encode-Decode
test_string = "Hello! How are you? ఏం చేస్తున్నావ్?"
utf8_encode = test_string.encode("utf-8")
print(utf8_encode)
print(type(utf8_encode))
list(utf8_encode) # Get the byte values for the encoded string (integers from 0 to 255).
# One byte does not necessarily correspond to one Unicode character!
print(len(test_string))
print(len(utf8_encode))
print(utf8_encode.decode(("utf-8")))

b'Hello! How are you? \xe0\xb0\x8f\xe0\xb0\x82 \xe0\xb0\x9a\xe0\xb1\x87\xe0\xb0\xb8\xe0\xb1\x8d\xe0\xb0\xa4\xe0\xb1\x81\xe0\xb0\xa8\xe0\xb1\x8d\xe0\xb0\xa8\xe0\xb0\xbe\xe0\xb0\xb5\xe0\xb1\x8d?'
<class 'bytes'>
36
64
Hello! How are you? ఏం చేస్తున్నావ్?
